In [1]:
import os
import mir_eval
import mir_eval.util as util
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import csv
import tqdm

def getAudioPaths(audio_txt):
    audiopaths = []
    with open(audio_txt, 'r') as file:
        for line in file.readlines():
            audiopaths.append(line.strip('\n'))

    return audiopaths

def getTBSnames(row):
    track_name = os.path.basename(row['trackname'])
    
    return os.path.basename(track_name)



In [6]:
date = '2025-04-04_MergedResults-dplp'
# pth_type = 'raw'
# pth_type = 'final'
main_dir = '/home/ALABSAD/sunnycyc/fsmm/WorkCC/dPLP/github_dPLPSFX/evaluation/2025-04-04_eval-dPLP/csvfiles/'

csv_savedir = os.path.join(main_dir, date)
csv_rpaths = glob.glob(os.path.join(main_dir, '*plpcompare*.csv'))

# csv_rpaths = glob.glob(os.path.join(main_dir, '2025-03*'))

if not os.path.exists(csv_savedir):
    Path(csv_savedir).mkdir(parents = True, exist_ok = True)


df_list = []
for csv_rpath in csv_rpaths:
    
    ### organize results 
    df_list.append( pd.read_csv(csv_rpath))
    
df = pd.concat(df_list)
datasets = list(df['Dataset'].unique())
print(len(df))

# print(df.keys())
df = df[['Dataset', 'plp_type', 'N', 'theta', 'trackname', 'Track',
       'F1', 'R', 'P', 'Onbeat-L2', 'Offbeat-L2', 'Double-L2', 'Triple-L2',
       'Quadruple-L2', 'Half-L2', 'Third-L2', 'Quarter-L2', 'Any-L2',
       'F-L2-onbeat', 'P-L2-onbeat', 'R-L2-onbeat', 'F-L2-half', 'P-L2-half',
       'R-L2-half', 'F-L2-third', 'P-L2-third', 'R-L2-third', 'F-L2-any',
       'P-L2-any', 'R-L2-any']]

plp_types = sorted(df['plp_type'].unique().tolist())
Ns = sorted(df['N'].unique().tolist())
thetas = sorted(df['theta'].unique().tolist())

print('plp_types:', plp_types)
print('Ns:', Ns)
print('thetas:', thetas)
# head_types = sorted(df['head_type'].unique().tolist())


# print('model_types:', model_types)
# print('head_type:', head_types)
df['tbsname'] = df.apply(getTBSnames, axis = 1)
trackbsnames = sorted(df['tbsname'].unique().tolist())


240
plp_types: ['A', 'S']
Ns: [3, 5, 10]
thetas: ['LG', 'LN']


In [9]:
train_info_dir = './dataset/gtzan/train-info/'
test_txt = os.path.join(train_info_dir, "test_files.txt")
test_tracks = getAudioPaths(test_txt)
test_tbs = [os.path.basename(i).replace('.wav', '.beats') for i in test_tracks]

df2 = df.loc[df['tbsname'].isin(test_tbs)]
# print(len(df2))
#%%
dataset = 'gtzan'
head_types = ['A-LG', 'A-LN', 'S-LG', 'S-LN']

head_rname_dict = {
    'dplp-N10':'dPLP-N10', 
    'dplp-N3':'dPLP-N3', 
    'dplp-N5':'dPLP-N5', 
    'fused':'F', 
    'spectral_flux':'S', 
    }


avg_list = []
std_list = []
def getAudioPaths(audio_txt):
    audiopaths = []
    with open(audio_txt, 'r') as file:
        for line in file.readlines():
            audiopaths.append(line.strip('\n'))
    return audiopaths

# unseen_list = []
train_info_dir = './dataset/gtzan/train-info/'
test_txt = os.path.join(train_info_dir, "test_files.txt")
test_tracks = getAudioPaths(test_txt)
test_tbs = [os.path.basename(i).replace('.wav', '.beats') for i in test_tracks]

df = df.loc[df['tbsname'].isin(test_tbs)]
# print(len(df))

# for head_type in [ 'spectral_flux', 'fused']:
for head_type in head_types:
    # break
    avg_list = []
    std_list = []
    plp_type, theta = head_type.split('-')
    # for fold_num in np.arange(0, 1):
        # break
        
    
        
    ### get df for corresponding models and trackname
    select_df = df2.loc[
        (df2['trackname'].isin(test_tbs))
                        & (df2['theta']==theta)
                        & (df2['plp_type']==plp_type)
                     ]
    # print(len(select_df))
    avg_list.append([select_df["F1"].mean(), select_df["P"].mean(), select_df["R"].mean(), 
                         select_df["F-L2-onbeat"].mean(), 
                         # select_df["F-L4-onbeat"].mean(), 
                         # select_df["F-L8-onbeat"].mean(), 
                         select_df["P-L2-onbeat"].mean(), 
                         # select_df["P-L4-onbeat"].mean(), 
                         # select_df["P-L8-onbeat"].mean(), 
                         select_df["R-L2-onbeat"].mean(), 
                         # select_df["R-L4-onbeat"].mean(), 
                         # select_df["R-L8-onbeat"].mean()
                         ]
                    
                    )
    std_list.append([select_df["F1"].std(), select_df["P"].std(), select_df["R"].std(), 
                     select_df["F-L2-onbeat"].std(), 
                     # select_df["F-L4-onbeat"].std(), 
                     # select_df["F-L8-onbeat"].std(), 
                     select_df["P-L2-onbeat"].std(), 
                     # select_df["P-L4-onbeat"].std(), 
                     # select_df["P-L8-onbeat"].std(), 
                     select_df["R-L2-onbeat"].std(), 
                     # select_df["R-L4-onbeat"].std(), 
                     # select_df["R-L8-onbeat"].std()
                     ]
                    
                    )
    avg_np = np.array(avg_list)
    avg_res = avg_np.mean(axis =0)
    std_np = np.array(std_list)
    std_res = std_np.mean(axis = 0)
       
    print( '{}'.format(  head_type) + \
          '& {:.3f} '.format(avg_res[0]) + \
            '& {:.3f} '.format(avg_res[1]) + \
            '& {:.3f} '.format(avg_res[2]) + \
            '& {:.3f} '.format(avg_res[3]) + \
            '& {:.3f} '.format(avg_res[4]) + \
            # '& {:.3f} '.format(avg_res[5]) + \
            # '& {:.3f} '.format(avg_res[6]) + \
            # '& {:.3f} '.format(avg_res[7]) + \
            # '& {:.3f} '.format(avg_res[8]) + \
            # '& {:.3f} '.format(avg_res[9]) + \
            # '& {:.3f} '.format(avg_res[10]) + \
            '& {:.3f}\\\\'.format(avg_res[5]) 
    
          )


240
A-LG& 0.663 & 0.532 & 0.922 & 0.170 & 0.153 & 0.196\\
A-LN& 0.662 & 0.527 & 0.928 & 0.157 & 0.140 & 0.183\\
S-LG& 0.670 & 0.551 & 0.906 & 0.195 & 0.182 & 0.218\\
S-LN& 0.664 & 0.528 & 0.932 & 0.166 & 0.148 & 0.193\\
